<a href="https://colab.research.google.com/github/FerdinandPyCode/ArtificialIntelligence/blob/main/Pipeline_Zero_Shot_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

File ‘bbc_text_cls.csv’ already there; not retrieving.



In [3]:
!pip install transformers

In [4]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import textwrap

from sklearn.metrics import roc_auc_score , f1_score , confusion_matrix

In [5]:
from transformers import pipeline

classifier = pipeline('zero-shot-classification')

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
classifier("This is a great movie", candidate_labels = ['positive','negative'])

{'sequence': 'This is a great movie',
 'labels': ['positive', 'negative'],
 'scores': [0.9969461560249329, 0.003053840948268771]}

In [7]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [8]:
len(df)

2225

In [9]:
df.sample(frac=1).head()

,text,labels
236,Britannia members' £42m windfall\n\nMore than ...,business
1612,Bell set for England debut\n\nBath prop Duncan...,sport
539,US critics laud comedy Sideways\n\nRoad trip c...,entertainment
1418,Middlesbrough 2-2 Charlton\n\nA late header by...,sport
526,Spark heads world Booker list\n\nDame Muriel S...,entertainment


In [11]:
labels = list(set(df['labels']))
labels

['politics', 'tech', 'sport', 'business', 'entertainment']

In [12]:
print(textwrap.fill(df.iloc[1024]['text']))

UK troops on Ivory Coast standby  Downing Street has confirmed British
troops are on standby in case they need to help evacuate several
hundred UK citizens from Ivory Coast.  The news came as it emerged
France had begun evacuating its hostages after days of anti-French
demonstrations. Trouble flared after nine French peacekeepers were
killed and President Jacques Chirac ordered the destruction of the
Ivory Coast's air force. A company of 100-120 UK troops is understood
to have been put on standby. They are ready to fly out and secure the
route from the embassy to the airport if called upon. Prime Minister
Tony Blair's official spokesman said: "There are British nationals in
Ivory Coast and as you can imagine we are making the usual contingency
plans in case the situation deteriorates further, but that's all I can
say at this stage."  He added the situation would be monitored
carefully in case evacuation became necessary. France has played a
lead role in international peacekeeping in Iv

In [13]:
df.iloc[1024]['labels']

'politics'

In [14]:
classifier(df.iloc[1024]['text'], candidate_labels = labels)

{'sequence': 'UK troops on Ivory Coast standby\n\nDowning Street has confirmed British troops are on standby in case they need to help evacuate several hundred UK citizens from Ivory Coast.\n\nThe news came as it emerged France had begun evacuating its hostages after days of anti-French demonstrations. Trouble flared after nine French peacekeepers were killed and President Jacques Chirac ordered the destruction of the Ivory Coast\'s air force. A company of 100-120 UK troops is understood to have been put on standby. They are ready to fly out and secure the route from the embassy to the airport if called upon. Prime Minister Tony Blair\'s official spokesman said: "There are British nationals in Ivory Coast and as you can imagine we are making the usual contingency plans in case the situation deteriorates further, but that\'s all I can say at this stage."\n\nHe added the situation would be monitored carefully in case evacuation became necessary. France has played a lead role in internati

In [ ]:
preds = classifier(df['text'].tolist(), candidate_labels = labels)

In [ ]:
preds

In [ ]:
predicted_labels = [d['labels'][0] for d in preds]

In [ ]:
df['predicted_labels'] = predicted_labels

In [ ]:
print("Acc:", np.mean(df['predicted_labels'] == df['labels']))

In [ ]:
N = len(df)
K = len(labels)
label2idx = {v:k for k,v in enumerate(labels)}

probs = np.zeros((N,K))
for i in range(N):

  # loop through labels and scores in corresponding order
  d = preds[i]
  for label,score in zip(d['labels'], d['scores']):
    k = label2idx[label]
    probs[i,k] = score



In [ ]:
int_labels = [label2idx[x] for x in df['labels']]

In [ ]:
int_preds = np.argmax(probs, axis = 1)
cm = confusion_matrix(int_labels, int_preds, normalize= 'true')

In [ ]:
def plot_cm(cm):
  df_cm = pd.DataFrame(cm,index=labels, columns=labels)
  ax = sn.heatmap(df_cm, annot=True , fmt ='.2g')
  ax.set_xlabel('Predicted')
  ax.set_ylabel('Target')

plot_cm(cm)

In [ ]:
f1_score(df['labels'], predicted_labels , average = 'micro')

In [ ]:
roc_auc_score(int_labels,probs,multi_class='ovo')